<div>Autor: <b>Patryk Wittbrodt</b><br><b>WZ_INIS6 50933</b></div> 

# Zadanie 2

### Przygotowanie

In [1]:
# import bibliotek
import sqlite3
import datetime

# tworzenie klasy, która będzie odpowiadać za operacje na bazie danych SQLite.
class DatabaseInterface:
    def __init__(self, database_path):
        self.conn = sqlite3.connect(database_path, detect_types=sqlite3.PARSE_DECLTYPES | sqlite3.PARSE_COLNAMES)
        self.cur = self.conn.cursor()

    # wstępne przygotowanie bazy - tworzenie tabeli, gdzie przechowywane będą informacje.
    def setup(self):
        createTableQuery = '''CREATE TABLE IF NOT EXISTS pets (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL,
            type TEXT NOT NULL,
            gender TEXT NOT NULL,
            dateOfBirth datetime,
            timeAdded timestamp
            )'''
        
        self.cur.execute(createTableQuery)

    # metoda na której będziemy opierać następne metody.
    def execute(self, query, data):
        self.cur.execute(query, data)
        self.conn.commit()
        return self.cur

    # C - create
    # dodawanie nowego pupila do bazy - czasami nie znamy jego daty urodzenia (np. gdy przygarniamy go z ulicy), więc przygotujemy metodę na dwa scenariusze.
    def addPet(self, name, type, gender, dateOfBirth=None):
        if dateOfBirth is not None:
            return self.execute("INSERT INTO pets (name, type, gender, dateOfBirth, timeAdded) VALUES (?, ?, ?, ?, ?)", (name, type, gender, dateOfBirth, datetime.datetime.now()))
        else:
            return self.execute("INSERT INTO pets (name, type, gender, dateOfBirth, timeAdded) VALUES (?, ?, ?, ?, ?)", (name, type, gender, None, datetime.datetime.now()))


    # R - read
    # podstawowe odczytywanie danych z bazy - aby wyświetlić wynik po wykonanej metodzie wykonujemy również metodę printSelectResults().
    def getPetsInfo(self):
        self.execute("SELECT * FROM pets", ())

    def getPetsByType(self, type):
        self.execute("SELECT * FROM pets WHERE type = ?", (type,))
    
    def getPetsByGender(self, gender):
        self.execute("SELECT * FROM pets WHERE gender = ?", (gender,))

    def getPetsBornIn(self, date):
        self.execute("SELECT * FROM pets WHERE dateOfBirth = ?", (date,))

    def getPetsBornBefore(self, date):
        self.execute("SELECT * FROM pets WHERE dateOfBirth < ?", (date,))

    def getPetsBornAfter(self, date):
        self.execute("SELECT * FROM pets WHERE dateOfBirth > ?", (date,))

    def getPetInfoByID(self, petID):
        return self.execute("SELECT * FROM pets WHERE id = ?", (petID,))

    def printSelectResults(self):
        records = self.cur.fetchall()

        for row in records:
            print("-------------------------")
            print("ID:", row[0])
            print("Nazwa:", row[1], "| typ:", row[2], "| płeć:", row[3])
            print("Data urodzenia: ", row[4])
            print("Data dodania: ", row[5])
        
        print("-------------------------")


    # U - update
    # zmiana wartości poszczególnych danych w bazie na podstawie ID pupila
    def changePetName(self, petID, newName):
        return self.execute("UPDATE pets SET name = ? WHERE id = ?", (newName, petID))
    
    def changePetType(self, petID, newType):
        return self.execute("UPDATE pets SET type = ? WHERE id = ?", (newType, petID))

    def changePetGender(self, petID, newGender):
        return self.execute("UPDATE pets SET gender = ? WHERE id = ?", (newGender, petID))

    def changePetDateOfBirth(self, petID, newDate):
        return self.execute("UPDATE pets SET dateOfBirth = ? WHERE id = ?", (newDate, petID))

    # metoda pozwalająca zmienić wszystkie dane od razu - stworzona tylko "dla zasady"
    def changePetAllData(self, petID, newName, newType, newGender, newDate):
        return self.execute("UPDATE pets SET name = ?, type = ?, gender = ?, dateOfBirth = ? WHERE id = ?", (newName, newType, newGender, newDate, petID))


    # D - delete
    # usuwanie pupila, pupilów i całej tabeli
    def deletePet(self, petID):
        return self.execute("DELETE FROM pets WHERE id = ?", (petID,))
    
    def deleteAllPets(self):
        return self.execute("DELETE FROM pets", ())

    def dropTable(self):
        self.cur.execute('DROP TABLE pets')

    def __del__(self):
        self.conn.close()

Po stworzeniu klasy, która znacznie ułatwi nam pracę na bazie, czas pokazać zastosowanie i efekty:

In [3]:
db = DatabaseInterface('pets.db') # łączymy się z bazą na podstawie ściezki do pliku i przypisujemy te połączenie do zmiennej.

db.setup() # przygotowujemy naszą bazę do przechowywania danych o pupilach - wystarczy wykonać tylko raz.


## CRUD
Create - tworzenie,  Read - odczytywanie, Update - aktualizowanie, Delete - usuwanie

### Tworzenie

Baza z początku będzie pusta, więc ją wypełnijmy dwoma przykładowymi pupilami i wyświetlimy zawartość bazy:

In [48]:
db.addPet('Nero', 'pies', 'samiec', datetime.date(2020, 10, 4)) # pies o imieniu Nero urodzony 4 października 2020 roku
db.addPet('Mruczek', 'kot', 'samiec', datetime.date(2018, 4, 20)) # kot o imieniu Mruczek urodzony 20 kwietnia 2018 roku

db.getPetsInfo() # pobieramy zawartość tabeli 'pets'
db.printSelectResults() # wyświetlamy wynik ostatniego selecta

-------------------------
ID: 1
Nazwa: Nero | typ: pies | płeć: samiec
Data urodzenia:  2020-10-04
Data dodania:  2022-02-27 17:04:52.796204
-------------------------
ID: 2
Nazwa: Mruczek | typ: kot | płeć: samiec
Data urodzenia:  2018-04-20
Data dodania:  2022-02-27 17:04:52.895206
-------------------------


Jak widać dodało nam dwa zwierzęta, każde ze swoim unikalnym ID. Na podstawie tego ID możemy przeprowadzać operacje takie jak wyszukiwanie danych, edytowanie ich bądź usunięcie zwierzaka z bazy. Zanim jednak do tego przejdziemy - spróbujmy do bazy dodać zwierzaka, którego daty urodzenia nie znamy:

In [49]:
db.addPet('Maniek', 'pies', 'samiec') # sami nadaliśmy mu imię i wiemy tylko tyle, zobaczymy co na to nasza baza danych.

db.getPetsInfo() # pobieramy zawartość tabeli 'pets'
db.printSelectResults() # wyświetlamy wynik ostatniego selecta

-------------------------
ID: 1
Nazwa: Nero | typ: pies | płeć: samiec
Data urodzenia:  2020-10-04
Data dodania:  2022-02-27 17:04:52.796204
-------------------------
ID: 2
Nazwa: Mruczek | typ: kot | płeć: samiec
Data urodzenia:  2018-04-20
Data dodania:  2022-02-27 17:04:52.895206
-------------------------
ID: 3
Nazwa: Maniek | typ: pies | płeć: samiec
Data urodzenia:  None
Data dodania:  2022-02-27 17:08:53.434958
-------------------------


Nasza baza nie ma problemu ze zwierzętami, których daty urodzenia nie znamy. Dla wyraźniejszych wyników testów dodajmy więcej zwierząt do bazy:

In [51]:
db.addPet('Azor', 'pies', 'samiec', datetime.date(2016, 7, 13))
db.addPet('Azor', 'pies', 'samiec', datetime.date(2016, 7, 13))
db.addPet('Azor', 'pies', 'samiec', datetime.date(2016, 7, 13))
db.addPet('Azor', 'pies', 'samiec', datetime.date(2016, 7, 13))
db.addPet('Azor', 'pies', 'samiec', datetime.date(2016, 7, 13))

db.getPetsInfo() # pobieramy zawartość tabeli 'pets'
db.printSelectResults() # wyświetlamy wynik ostatniego selecta

-------------------------
ID: 1
Nazwa: Nero | typ: pies | płeć: samiec
Data urodzenia:  2020-10-04
Data dodania:  2022-02-27 17:04:52.796204
-------------------------
ID: 2
Nazwa: Mruczek | typ: kot | płeć: samiec
Data urodzenia:  2018-04-20
Data dodania:  2022-02-27 17:04:52.895206
-------------------------
ID: 3
Nazwa: Maniek | typ: pies | płeć: samiec
Data urodzenia:  None
Data dodania:  2022-02-27 17:08:53.434958
-------------------------
ID: 4
Nazwa: Azor | typ: pies | płeć: samiec
Data urodzenia:  2016-07-13
Data dodania:  2022-02-27 17:13:49.041855
-------------------------
ID: 5
Nazwa: Azor | typ: pies | płeć: samiec
Data urodzenia:  2016-07-13
Data dodania:  2022-02-27 17:13:49.136862
-------------------------
ID: 6
Nazwa: Azor | typ: pies | płeć: samiec
Data urodzenia:  2016-07-13
Data dodania:  2022-02-27 17:13:49.236855
-------------------------
ID: 7
Nazwa: Azor | typ: pies | płeć: samiec
Data urodzenia:  2016-07-13
Data dodania:  2022-02-27 17:13:49.344843
---------------

### Usuwanie
Dodaliśmy duplikaty Azora. Załóżmy, że ktoś zrobił to przez przypadek i teraz chcemy je usunąć. Wystarczy się odwołać do ID zwierzaka, którego chcemy usunąć z bazy. Jeśli chcemy usuwać wiele zwierząt można zrobić to za pomocą pętli która będzie działać na tablicy, gdzie podane są ID do usunięcia. Z uwagi na małą ilość "Azorów" usunę jednego po drugim, linijka po linijce:

In [53]:
# pierwszy Azor ma ID 4, czyli tego nie usuwamy - jest to oryginalny Azor
db.deletePet(5)
db.deletePet(6)
db.deletePet(7)
db.deletePet(8)

db.getPetsInfo() # pobieramy zawartość tabeli 'pets'
db.printSelectResults() # wyświetlamy wynik ostatniego selecta

-------------------------
ID: 1
Nazwa: Nero | typ: pies | płeć: samiec
Data urodzenia:  2020-10-04
Data dodania:  2022-02-27 17:04:52.796204
-------------------------
ID: 2
Nazwa: Mruczek | typ: kot | płeć: samiec
Data urodzenia:  2018-04-20
Data dodania:  2022-02-27 17:04:52.895206
-------------------------
ID: 3
Nazwa: Maniek | typ: pies | płeć: samiec
Data urodzenia:  None
Data dodania:  2022-02-27 17:08:53.434958
-------------------------
ID: 4
Nazwa: Azor | typ: pies | płeć: samiec
Data urodzenia:  2016-07-13
Data dodania:  2022-02-27 17:13:49.041855
-------------------------


Teraz możemy przystąpić do prawdziwego dodawania zwierzaków:

In [54]:
db.addPet('Sonia', 'pies', 'samica', datetime.date(2016, 7, 15))
db.addPet('Nela', 'kot', 'samica', datetime.date(2013, 1, 22))
db.addPet('Pelagia', 'papuga', 'samica', datetime.date(2019, 10, 5))
db.addPet('Luna', 'pies', 'samica', datetime.date(2021, 3, 10))
db.addPet('Yuri', 'jaszczurka', 'samiec', datetime.date(2021, 4, 1))
db.addPet('Babi', 'pies', 'samiec', datetime.date(2021, 4, 10))
db.addPet('Kacper', 'kot', 'samiec', datetime.date(2020, 3, 1))

### Odczytywanie

Na ten moment zwierzaków jest już więcej, więc wyświetlanie pełnej listy nie ma większego sensu - już sprawdziliśmy działanie tej metody. Oprócz wyświetlania wszystkich zwierząt metody z naszej klasy oferują nam wyświetlanie na podstawie typu (rodzaju) zwięrząt, na podstawie płci czy daty urodzenia (dokładny dzień, urodzone przed i urodzone po). Sprawdźmy to:

Wyświetlmy tylko psy:

In [55]:
db.getPetsByType('pies')
db.printSelectResults()

-------------------------
ID: 1
Nazwa: Nero | typ: pies | płeć: samiec
Data urodzenia:  2020-10-04
Data dodania:  2022-02-27 17:04:52.796204
-------------------------
ID: 3
Nazwa: Maniek | typ: pies | płeć: samiec
Data urodzenia:  None
Data dodania:  2022-02-27 17:08:53.434958
-------------------------
ID: 4
Nazwa: Azor | typ: pies | płeć: samiec
Data urodzenia:  2016-07-13
Data dodania:  2022-02-27 17:13:49.041855
-------------------------
ID: 9
Nazwa: Sonia | typ: pies | płeć: samica
Data urodzenia:  2016-07-15
Data dodania:  2022-02-27 17:25:37.660803
-------------------------
ID: 12
Nazwa: Luna | typ: pies | płeć: samica
Data urodzenia:  2021-03-10
Data dodania:  2022-02-27 17:25:38.425924
-------------------------
ID: 14
Nazwa: Babi | typ: pies | płeć: samiec
Data urodzenia:  2021-04-10
Data dodania:  2022-02-27 17:25:38.609348
-------------------------


Wyświetlmy tylko papugi:

In [56]:
db.getPetsByType('papuga')
db.printSelectResults()

-------------------------
ID: 11
Nazwa: Pelagia | typ: papuga | płeć: samica
Data urodzenia:  2019-10-05
Data dodania:  2022-02-27 17:25:38.317800
-------------------------


Wszystko póki co działa jak powinno. Teraz wyświetlmy wszystkie samice:

In [57]:
db.getPetsByGender('samica')
db.printSelectResults()

-------------------------
ID: 9
Nazwa: Sonia | typ: pies | płeć: samica
Data urodzenia:  2016-07-15
Data dodania:  2022-02-27 17:25:37.660803
-------------------------
ID: 10
Nazwa: Nela | typ: kot | płeć: samica
Data urodzenia:  2013-01-22
Data dodania:  2022-02-27 17:25:38.221801
-------------------------
ID: 11
Nazwa: Pelagia | typ: papuga | płeć: samica
Data urodzenia:  2019-10-05
Data dodania:  2022-02-27 17:25:38.317800
-------------------------
ID: 12
Nazwa: Luna | typ: pies | płeć: samica
Data urodzenia:  2021-03-10
Data dodania:  2022-02-27 17:25:38.425924
-------------------------


Teraz sprawdźmy jaki zwierzak urodził się 22 stycznia 2013 roku:

In [4]:
db.getPetsBornIn(datetime.date(2013, 1, 22))
db.printSelectResults()

-------------------------
ID: 10
Nazwa: Nela | typ: kot | płeć: samica
Data urodzenia:  2013-01-22
Data dodania:  2022-02-27 17:25:38.221801
-------------------------


A co jeśli podamy dzień, w którym nie urodził się żaden pupil?

In [5]:
db.getPetsBornIn(datetime.date(2010, 1, 1))
db.printSelectResults()

-------------------------


Dostaliśmy po prostu zakończenie wyświetlania (kosmetyczne). Wyświetlmy teraz listę wszystkich zwierzaków, które urodziły się przed 15 lipca 2016 roku:

In [6]:
db.getPetsBornBefore(datetime.date(2016, 7, 15))
db.printSelectResults()

-------------------------
ID: 4
Nazwa: Azor | typ: pies | płeć: samiec
Data urodzenia:  2016-07-13
Data dodania:  2022-02-27 17:13:49.041855
-------------------------
ID: 10
Nazwa: Nela | typ: kot | płeć: samica
Data urodzenia:  2013-01-22
Data dodania:  2022-02-27 17:25:38.221801
-------------------------


Zwierzaki urodzone po 20 stycznia 2020 roku:

In [7]:
db.getPetsBornAfter(datetime.date(2020, 1, 20))
db.printSelectResults()

-------------------------
ID: 1
Nazwa: Nero | typ: pies | płeć: samiec
Data urodzenia:  2020-10-04
Data dodania:  2022-02-27 17:04:52.796204
-------------------------
ID: 12
Nazwa: Luna | typ: pies | płeć: samica
Data urodzenia:  2021-03-10
Data dodania:  2022-02-27 17:25:38.425924
-------------------------
ID: 13
Nazwa: Yuri | typ: jaszczurka | płeć: samiec
Data urodzenia:  2021-04-01
Data dodania:  2022-02-27 17:25:38.509417
-------------------------
ID: 14
Nazwa: Babi | typ: pies | płeć: samiec
Data urodzenia:  2021-04-10
Data dodania:  2022-02-27 17:25:38.609348
-------------------------
ID: 15
Nazwa: Kacper | typ: kot | płeć: samiec
Data urodzenia:  2020-03-01
Data dodania:  2022-02-27 17:25:38.709244
-------------------------


Ostatnim z wyświetlania będzie pokazanie wyświetlania na podstawie ID, o którym już wcześniej wspomniałem:

In [8]:
db.getPetInfoByID(13) # ID jaszczurki
db.printSelectResults()

-------------------------
ID: 13
Nazwa: Yuri | typ: jaszczurka | płeć: samiec
Data urodzenia:  2021-04-01
Data dodania:  2022-02-27 17:25:38.509417
-------------------------


### Aktualizowanie
Dane zwierzaków możemy zmienić na podstawie ID. Wyżej wyświetliliśmy dane jaszczurki - zmieńmy jej imię i sprawdźmy rezultaty:

In [9]:
db.changePetName(13, 'Ivan')

db.getPetInfoByID(13) # ID jaszczurki
db.printSelectResults()

-------------------------
ID: 13
Nazwa: Ivan | typ: jaszczurka | płeć: samiec
Data urodzenia:  2021-04-01
Data dodania:  2022-02-27 17:25:38.509417
-------------------------


Jak widać zmieniliśmy nazwę z 'Yuri' na 'Ivan'. Załóżmy teraz, że ktoś przez pomyłkę wpisał, że nasz 'Kacper' (ID 15) nie jest kotem, tylko psem. Naprawmy to:

In [10]:
db.changePetType(15, 'pies')

db.getPetInfoByID(15) # ID Kacpra
db.printSelectResults()

-------------------------
ID: 15
Nazwa: Kacper | typ: pies | płeć: samiec
Data urodzenia:  2020-03-01
Data dodania:  2022-02-27 17:25:38.709244
-------------------------


Oprócz zmiany imienia czy typu zwierzaka są też metody na zmianę płci czy daty urodzenia - działają na tym samym schemacie. Istnieje też metoda która pozwala na zmianę wszystkich danych jednocześnie:

In [11]:
db.changePetAllData(15, 'Chmurka', 'pies', 'samica', datetime.date(2021, 11, 11)) # Kacper "magicznie" zmienił się w Chmurkę

db.getPetInfoByID(15) # ID Kacpra, teraz już Chmurki
db.printSelectResults()

-------------------------
ID: 15
Nazwa: Chmurka | typ: pies | płeć: samica
Data urodzenia:  2021-11-11
Data dodania:  2022-02-27 17:25:38.709244
-------------------------
